In [2]:
! pip install langchain_openai langchain_core langchain -U

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.4/63.4 kB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.5/438.5 kB 17.2 MB/s eta 0:00:00
  Attempting uninstall: langchain_core
    Found existing installation: langchain-core 0.3.60
    Uninstalling langchain-core-0.3.60:
      Successfully uninstalled langchain-core-0.3.60


# Output parsers
The information here refers to parsers that take a text output from a model try to parse it into a more structured representation.

 More and more models are supporting **function (or tool) calling**, which handles this automatically.

 It is recommended to use function/tool calling rather than output parsing

# JSON Mode

在普通llm调用中，生成的结果没法直接使用在程序中，因为程序一般需要和明确的数据结构交互，而大模型给出的结果往往是非结构化的。

> 假设我们有一个获取获取某地天气的方法get_weather，当我们询问北京的天气时，需要让大模型从用户意图中提取出北京，再输出get_weather需要的格式，最后调用get_weather方法。

```python
def get_weather(params):
    # 从字典中提取 location 参数
    location = params.get('location')    
    return location + "的天气是晴天"
```

OpenAI的JSON模式（JSON Mode）是一种在生成文本时，要求模型输出格式为JSON的特殊模式。

这种模式通常用于需要结构化输出的场景，例如需要模型生成特定格式的数据、配置文件、API响应等。

1. 结构化输出：在JSON模式下，模型生成的文本必须是有效的JSON格式。这意味着输出将包含**键值对、数组、对象等结构化数据**，而不是自由文本。
2. 明确性：由于JSON是一种严格的格式，模型在生成输出时必须遵循**预定义**的结构，这有助于减少歧义，确保输出的内容符合预期。
3. 可解析性：**生成的JSON可以直接被解析和处理**，便于后续的**程序化操作**。例如，可以直接将生成的JSON数据加载到**Python的字典或列表**中，进行进一步的处理。

In [3]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain.output_parsers.json import SimpleJsonOutputParser
from google.colab import userdata

import os

os.environ["OPENAI_API_BASE"] = userdata.get('OPENAI_API_BASE')
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

def get_weather(params):
    # 从字典中提取 location 参数
    location = params.get('location')
    return location + "的天气是晴天"

model = ChatOpenAI(
    model="qwen2.5-72b-instruct",
    model_kwargs={ "response_format": { "type": "json_object" } },
)

prompt = ChatPromptTemplate.from_template(
    "获取用户问题中提到的城市"
    '你必须始终输出一个包含‘location’键 JSON 对象。'
    "用户问题：{question}"
)

chain = prompt | model | SimpleJsonOutputParser()

res = chain.invoke({ "question": "北京的天气怎么样" })

print(res)
print(type(res))
print()
print(get_weather(res))


{'location': '北京'}
<class 'dict'>

北京的天气是晴天


JsonOutputParser

kwargs (Any | None) – Additional keyword arguments to pass to the Runnable.
``` python
model_kwargs={ "response_format": { "type": "json_object" } },
```




上面的写法，虽然可以实现把用户的输入转化成结构化的数据，但是可以明显看出以下弊端：
- 你需要写**大量的提示词**，来描述生成的json包含字段，尽管如此，大模型还是**有概率误解你的意图**，导致生成的**json数据不符合规范**，而且很依赖大模型本身能力。
- 对于想要生成**特别复杂的json结构的场景**，提示词也会变得非常庞大，生成的结构也更加不可控。
对于上述存在的问题，我们可以使用大模型提供的一个非常棒的能力：function calling。